# Checking the top 100 - Market Cap
In this notebook we will create an Excel file of the 100 crypto's with the biggest maket cap. After this we can create a new XLS file with the same code created. Now we can compare the two excel files to see the differences. These differences will be the new crypto currencies that just entered the top 100 coins with market cap.

### The libraries we use
In this code-block I show all the libraries I use in my python code. If you do not have one of these just install them by typing !pip install 'library name'

In [95]:
#We use beatifulsoup to parse the text from the url we use.
from bs4 import BeautifulSoup
import justext
#We need the time to append a time at our backup so we are sure when it was created
from time import gmtime, strftime
#We need pandas to store the data in the pandas object so we can easily create an excel file. 
import pandas as pd
#we need OS to delete the Excel file in the folder if it already excists
import os

### Script 1 - Creating a Dataframe of the data shown on coinmarketcap
In the first script we will parse the data that is contained in the table on the website named https://coinmarketcap.com/currencies'. We will store this data in a pandas dataframe.

In [103]:
def DownloadCurrencies():
    '''This function first creates an inner list from the data gathered at the following webpage:
    'https://coinmarketcap.com/currencies'. It downloads the arrtibutes of the 100 currencies with
    the biggest market cap. After this it stores the inner list in a pandas object.'''
    
    inner_list = []
    url = 'https://coinmarketcap.com/currencies'

    r  = requests.get(url)
    data = r.text
    soup = BeautifulSoup(data)

    #Using a triple for loop we can reach the valueble information stored in the table.
    for table in soup.find_all('tbody'):
        for row in table.find_all('tr'):
            #We will store the data of one currency in this list.
            one_row = []
            for text in row.find_all('td'):
                one_row.append(text.text.strip())
            inner_list.append(one_row)
    
    #Creating the column names and setting the index as the name of the currency.
    columns= ["Rank","Name","Market cap","Price","Circulating Supply","Volume","% Change 24h", "Price graph"]            
    final_file = pd.DataFrame(inner_list, columns=columns)
    final_file = final_file.drop("Price graph", 1)
    #final_file.set_index('Name', inplace=True)
    
    return final_file

### Script 2 - Create the excel file + timestamp
In this code block we use the script created above. We transform the pandas dataframe to a excel file and store it in the same folder as this notebook. If you run this script for the second time it will overwrite the excel file you created earlier. We also create a timestamp to be certain when it was created and how old your comparison file  is.


In [97]:
## WARNING: by running this script you create a new backup file! ##

def CreateBackup():
    """By running this code you create a new backup. This will be the Excel file the script compares with to 
    check for new entries in the top 100."""
    #Here we create an Excel file with the 100 data stored within it
    writer = pd.ExcelWriter('top100.xlsx')
    #We use a try/except in case we want to overwrite the excel file we created earlier.
    try:
        DownloadCurrencies().to_excel(writer,'top100')
        writer.save()
    except IOError:
        os.remove('top100.xlsx')
        DownloadCurrencies().to_excel(writer,'top100')
        writer.save()
        
    #Here we create a simple text file containing the timestamp
    timestamp = strftime("%Y-%m-%d %H:%M:%S", gmtime())
    file = open("testfile", "w")
    file.write(timestamp)
    file.close()
    
CreateBackup()

### Script 3 -  Comparing the current top 100 cryptocurrencies with the back-up file we created earlier.
This last script compares the backup excel file with the current state of the webpage. It prints all the new crypto's that just entered the top 100 in market cap!

In [140]:
def NewTop100():
    CurrentState = DownloadCurrencies()
    backup = pandas.read_excel('top100.xlsx')
    
    #Creating 2 lists with all the cryto names in both files.
    Current_cryptos = [name for name in CurrentState['Name']]
    backup_cryptos = [name for name in backup['Name']]

    new_cryptos = []
    for name in Current_cryptos:
        if name not in backup_cryptos:
            new_cryptos.append(name)
#CurrentState['Name']

    final_df = CurrentState.loc[CurrentState['Name'].isin(new_cryptos)]
    
    #Also printing the time the backup was created so we know how old it is!
    file_object = open("testfile", "r")
    The_Time = file_object.read()
    
    print "The new cryptos that entered the top 100 in marketcap since your last backup created on", The_Time, "are stored in the dataframe below."
    return final_df
    
NewTop100()

The new cryptos that entered the top 100 in marketcap since your last backup created on 2017-07-06 17:51:54 are stored in the dataframe below.


,Rank,Name,Market cap,Price,Circulating Supply,Volume,% Change 24h
99,100,Rubycoin,"$11,889,380",$0.484978,"24,515,298\n RBY \n...","$10,337",-1.41%
